<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/tests_test_symbolic_theory_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# tests/test_symbolic_theory.py
import sympy as sp
from symbolic_theory_suite import (
    SymbolicTheoryConstructor,
    BuildConfig,
    SolveConfig,
    _build_demo_graph,
)

def test_build_edges_only_counts():
    ctor = SymbolicTheoryConstructor(var_prefix="v")
    G = _build_demo_graph()
    theory = ctor.build(G, BuildConfig(restrict_to_edges=True, add_constant=True, var_prefix="v"))
    eqs = theory["equations"]
    assert len(eqs) == 2  # x-y, y-z
    s = ctor.pretty_print(theory)
    assert "a_x_y" in s and "a_y_z" in s

def test_build_all_pairs_counts():
    ctor = SymbolicTheoryConstructor(var_prefix="v")
    G = _build_demo_graph()
    theory = ctor.build(G, BuildConfig(restrict_to_edges=False, add_constant=True, var_prefix="v"))
    eqs = theory["equations"]
    assert len(eqs) == 3  # x-y, x-z, y-z

def test_edges_only_unique_solution_with_anchor():
    ctor = SymbolicTheoryConstructor(var_prefix="v")
    G = _build_demo_graph()
    theory = ctor.build(G, BuildConfig(restrict_to_edges=True, add_constant=True, var_prefix="v"))
    res = ctor.try_solve(
        theory,
        SolveConfig(coeff_value=1.0, const_value=0.0, anchors={"v_x": 1.0}, max_equations=10),
    )
    assert res["status"] in ("unique", "anchored_only")
    # Expect v_y = -1, v_z = 1 under unit coefficients and v_x=1
    first = res["assignments"][0]
    assert float(first["v_y"]) == -1.0
    assert float(first["v_z"]) == 1.0

def test_all_pairs_auto_anchor_to_unique():
    ctor = SymbolicTheoryConstructor(var_prefix="v")
    G = _build_demo_graph()
    theory = ctor.build(G, BuildConfig(restrict_to_edges=False, add_constant=True, var_prefix="v"))
    res = ctor.try_solve(
        theory,
        SolveConfig(coeff_value=1.0, const_value=0.0, anchors={"v_x": 1.0}, auto_anchor=True, auto_anchor_value=1.0),
    )
    # With auto-anchoring, system should become unique or fully anchored
    assert res["status"] in ("unique", "anchored_only")
    assert "auto_anchor_steps" in res